In [1]:
import numpy as np
import pandas as pd

# get data
# ! conda install pandas-datareader s
import pandas_datareader as pdr

# visual
# ! pip install mpl-finance
import matplotlib.pyplot as plt
import mpl_finance as mpf

import seaborn as sns

#time
from datetime import datetime
from datetime import timedelta

# https://github.com/mrjbq7/ta-lib
# ! pip install ta-lib
import talib

In [2]:
start_date = datetime(2018,1,1)     #開始日期
end_date= datetime(2018,12,31)      #結束日期

In [3]:
df_TWSE2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv",encoding="utf-8")
df_TWSE2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv",encoding="utf-8")
df_TWSE2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv",encoding="utf-8")

In [4]:
df_TWSE = pd.concat([df_TWSE2016, df_TWSE2017, df_TWSE2018])  #三個合併
del df_TWSE2016, df_TWSE2017, df_TWSE2018

In [5]:
# ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '成交值(千元)', '成交筆數(筆)', '流通在外股數(千股)', '本益比-TSE', '股價淨值比-TSE']
df_TWSE['證券代碼'] = df_TWSE['證券代碼'].astype(str)
df_TWSE['年月日'] = pd.to_datetime(df_TWSE['年月日'])
df_TWSE['開盤價(元)'] = df_TWSE['開盤價(元)'].str.replace(',' , '').astype('float64') # 1,000 to 1000 to float
df_TWSE['最高價(元)'] = df_TWSE['最高價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['最低價(元)'] = df_TWSE['最低價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['收盤價(元)'] = df_TWSE['收盤價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交量(千股)'] = df_TWSE['成交量(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['成交值(千元)'] = df_TWSE['成交值(千元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交筆數(筆)'] = df_TWSE['成交筆數(筆)'].str.replace(',' , '').astype('int64')
df_TWSE['流通在外股數(千股)'] = df_TWSE['流通在外股數(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['本益比-TSE'] = df_TWSE['本益比-TSE'].str.replace(',' , '').astype('float64')
df_TWSE['股價淨值比-TSE'] = df_TWSE['股價淨值比-TSE'].astype('float64')
df_TWSE.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016-12-30,29.09,29.64,28.93,29.64,6121.0,213121.0,2939,3692175.0,20.56,1.23
1,1102 亞泥,2016-12-30,24.91,24.91,24.67,24.81,3347.0,88078.0,1227,3361447.0,31.75,0.73
2,1103 嘉泥,2016-12-30,8.29,8.29,8.29,8.29,0.0,0.0,0,774780.0,11.19,0.40
3,1104 環泥,2016-12-30,21.19,21.54,21.19,21.54,141.0,3492.0,115,634572.0,9.27,0.97
4,1108 幸福,2016-12-30,8.55,8.60,8.53,8.55,90.0,822.0,33,404738.0,10.99,0.78


In [6]:
#df_2327 = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True)
#df_2456 = df_TWSE[df_TWSE['證券代碼'].str.contains('奇力新')].sort_values(by=['年月日']).reset_index(drop=True)
#df_2478 = df_TWSE[df_TWSE['證券代碼'].str.contains('大毅')].sort_values(by=['年月日']).reset_index(drop=True)
#df_6271 = df_TWSE[df_TWSE['證券代碼'].str.contains('同欣電')].sort_values(by=['年月日']).reset_index(drop=True)
df_2327 = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True) # 用日期排序 再把index重排
del df_TWSE

In [7]:
df_2327.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36
2,2327 國巨,2016-01-06,68.36,69.40,67.84,68.10,1751.0,92198.0,692,651594.0,9.31,1.36
3,2327 國巨,2016-01-07,66.81,68.62,66.81,68.10,992.0,52047.0,657,651594.0,9.31,1.36
4,2327 國巨,2016-01-08,68.10,69.27,67.71,69.27,1156.0,61362.0,598,651594.0,9.47,1.38


In [8]:
# df_2327['Date']=df_2327.index             # index 複製到另一欄
# df_2327 = df_2327.reset_index(drop=True)  # index變成 0~N
# df_2327.head()

In [9]:
##增欄位:fluctuation幅度 tag漲跌平
df_2327['fluctuation'] = 0.0
df_2327['tag']='--'
df_2327.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,0.0,--
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,0.0,--
2,2327 國巨,2016-01-06,68.36,69.40,67.84,68.10,1751.0,92198.0,692,651594.0,9.31,1.36,0.0,--
3,2327 國巨,2016-01-07,66.81,68.62,66.81,68.10,992.0,52047.0,657,651594.0,9.31,1.36,0.0,--
4,2327 國巨,2016-01-08,68.10,69.27,67.71,69.27,1156.0,61362.0,598,651594.0,9.47,1.38,0.0,--


In [10]:
###計算漲跌
for index, row in df_2327.iterrows():
    try:
        margin =(float(df_2327.loc[index,'收盤價(元)']) - float(df_2327.loc[index-1,'收盤價(元)']) )/ float(df_2327.loc[index-1,'收盤價(元)'])
        df_2327.loc[index,'fluctuation']=margin
        if margin >=0.03:
            df_2327.loc[index,'tag']='up'
        elif margin <= -0.03:
            df_2327.loc[index,'tag']='down'
        else:
            df_2327.loc[index,'tag']='even'
    except:
        continue

In [11]:
df_2327.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,0.000000,--
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,-0.003789,even
2,2327 國巨,2016-01-06,68.36,69.40,67.84,68.10,1751.0,92198.0,692,651594.0,9.31,1.36,-0.003803,even
3,2327 國巨,2016-01-07,66.81,68.62,66.81,68.10,992.0,52047.0,657,651594.0,9.31,1.36,0.000000,even
4,2327 國巨,2016-01-08,68.10,69.27,67.71,69.27,1156.0,61362.0,598,651594.0,9.47,1.38,0.017181,even


In [12]:
up_fliter=(df_2327['tag']=='up')
up_2327=df_2327[up_fliter]
len(up_2327)

114

In [13]:
down_fliter=(df_2327['tag']=='down')
down_2327=df_2327[down_fliter]
len(down_2327)

89

In [14]:
df_2327['fluctuation'] = 0.0
df_2327['tag']='--'
df_2327['closeshift'] = 0.0
df_2327.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,0.0,--,0.0
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,0.0,--,0.0
2,2327 國巨,2016-01-06,68.36,69.40,67.84,68.10,1751.0,92198.0,692,651594.0,9.31,1.36,0.0,--,0.0
3,2327 國巨,2016-01-07,66.81,68.62,66.81,68.10,992.0,52047.0,657,651594.0,9.31,1.36,0.0,--,0.0
4,2327 國巨,2016-01-08,68.10,69.27,67.71,69.27,1156.0,61362.0,598,651594.0,9.47,1.38,0.0,--,0.0


In [15]:
df_2327['closeshift'] = df_2327['收盤價(元)'].shift(periods=1)#.fillna(value=0.0, inplace=True)
#df_2327['closeshift'] = df_2327['closeshift'].fillna(value= 0.0, inplace=True)
df_2327.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,0.0,--,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,0.0,--,68.62
2,2327 國巨,2016-01-06,68.36,69.40,67.84,68.10,1751.0,92198.0,692,651594.0,9.31,1.36,0.0,--,68.36
3,2327 國巨,2016-01-07,66.81,68.62,66.81,68.10,992.0,52047.0,657,651594.0,9.31,1.36,0.0,--,68.10
4,2327 國巨,2016-01-08,68.10,69.27,67.71,69.27,1156.0,61362.0,598,651594.0,9.47,1.38,0.0,--,68.10


In [16]:
df_2327['fluctuation'] = (df_2327['收盤價(元)'] - df_2327['closeshift']) / df_2327['closeshift']
df_2327.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,NaN,--,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,-0.003789,--,68.62
2,2327 國巨,2016-01-06,68.36,69.40,67.84,68.10,1751.0,92198.0,692,651594.0,9.31,1.36,-0.003803,--,68.36
3,2327 國巨,2016-01-07,66.81,68.62,66.81,68.10,992.0,52047.0,657,651594.0,9.31,1.36,0.000000,--,68.10
4,2327 國巨,2016-01-08,68.10,69.27,67.71,69.27,1156.0,61362.0,598,651594.0,9.47,1.38,0.017181,--,68.10


In [17]:
df_2327['tag']='even'
df_2327.loc[df_2327['fluctuation'] >=  0.03, 'tag'] = 'up'
df_2327.loc[df_2327['fluctuation'] <= -0.03, 'tag'] = 'down'
df_2327.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,NaN,even,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,-0.003789,even,68.62
2,2327 國巨,2016-01-06,68.36,69.40,67.84,68.10,1751.0,92198.0,692,651594.0,9.31,1.36,-0.003803,even,68.36
3,2327 國巨,2016-01-07,66.81,68.62,66.81,68.10,992.0,52047.0,657,651594.0,9.31,1.36,0.000000,even,68.10
4,2327 國巨,2016-01-08,68.10,69.27,67.71,69.27,1156.0,61362.0,598,651594.0,9.47,1.38,0.017181,even,68.10


In [18]:
up_2327=df_2327[df_2327['tag']=='up']
len(up_2327)

114

In [19]:
down_2327=df_2327[df_2327['tag']=='down']
len(down_2327)

89

In [20]:
df_forum = pd.read_csv("bda2019_dataset/forum2.csv",encoding="utf-8")
df_forum.head()

,id,p_type,s_name,s_area_name,comment_count,post_time,title,author,content,page_url
0,1451585357153_F01,forum,mobile01,投資與理財(生活娛樂),1,2016/1/1 2:01:00,自刪,famas2200,不小心發錯版主請幫忙刪除文章謝謝,http://www.mobile01.com/topicdetail.php?f=291&...
1,1451616871574_F01,forum,mobile01,投資與理財(生活娛樂),5,2016/1/1 10:41:00,傳超微已打算將訂單下給格羅方德,taiwan2008,台積電爭奪超微下一代繪圖處理器失利，因為傳超微已打算將訂單下給格羅方德（GlobalFoun...,http://www.mobile01.com/topicdetail.php?f=291&...
2,1451616872738_F01,forum,mobile01,投資與理財(生活娛樂),4,2016/1/1 10:45:00,蘋果將另尋成長動能發動購併 ??,taiwan2008,有分析師認為 蘋果將另尋成長動能，或許會動用深到不行的口袋，發動購併，可能目標包括 Adob...,http://www.mobile01.com/topicdetail.php?f=291&...
3,1451618546867_F01,forum,mobile01,投資與理財(生活娛樂),7,2016/1/1 11:19:00,[詢問] 停徵證所稅,Go Go Free Life,新聞相關報導:<BR>[立法院院會去年11月三讀修正通過所得稅法部分條文，刪除有關證所稅相關...,http://www.mobile01.com/topicdetail.php?f=291&...
4,1451623256525_F01,forum,mobile01,投資與理財(生活娛樂),7,2016/1/1 12:29:00,網路出金,mobiliken,請問一下，網路出金是什麼？謝謝各位喔。,http://www.mobile01.com/topicdetail.php?f=291&...


In [21]:
len(df_forum)

25952

In [22]:
df_forum['post_time'] = pd.to_datetime(df_forum['post_time'])
df_forum['post_time2'] = df_forum['post_time'].dt.date   # .dt.date用在dataframe  .date()用在一個
#df_forum['label'] = 'even'
df_forum.head()

,id,p_type,s_name,s_area_name,comment_count,post_time,title,author,content,page_url,post_time2
0,1451585357153_F01,forum,mobile01,投資與理財(生活娛樂),1,2016-01-01 02:01:00,自刪,famas2200,不小心發錯版主請幫忙刪除文章謝謝,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-01
1,1451616871574_F01,forum,mobile01,投資與理財(生活娛樂),5,2016-01-01 10:41:00,傳超微已打算將訂單下給格羅方德,taiwan2008,台積電爭奪超微下一代繪圖處理器失利，因為傳超微已打算將訂單下給格羅方德（GlobalFoun...,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-01
2,1451616872738_F01,forum,mobile01,投資與理財(生活娛樂),4,2016-01-01 10:45:00,蘋果將另尋成長動能發動購併 ??,taiwan2008,有分析師認為 蘋果將另尋成長動能，或許會動用深到不行的口袋，發動購併，可能目標包括 Adob...,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-01
3,1451618546867_F01,forum,mobile01,投資與理財(生活娛樂),7,2016-01-01 11:19:00,[詢問] 停徵證所稅,Go Go Free Life,新聞相關報導:<BR>[立法院院會去年11月三讀修正通過所得稅法部分條文，刪除有關證所稅相關...,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-01
4,1451623256525_F01,forum,mobile01,投資與理財(生活娛樂),7,2016-01-01 12:29:00,網路出金,mobiliken,請問一下，網路出金是什麼？謝謝各位喔。,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-01


In [23]:
df_2327.loc[2,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[1,'年月日'].date()

True

In [24]:
df_2327.loc[5,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[4,'年月日'].date()

False

In [25]:
df_2327.loc[0,'年月日'].date() 

datetime.date(2016, 1, 4)

In [26]:
df_forum.loc[50,'post_time'].date()

datetime.date(2016, 1, 4)

In [27]:
df_forum[ df_forum['post_time2'] == df_2327.loc[0,'年月日'].date() ].head()

,id,p_type,s_name,s_area_name,comment_count,post_time,title,author,content,page_url,post_time2
39,1451838210829_F01,forum,mobile01,投資與理財(生活娛樂),14,2016-01-04 00:21:00,"請問關於持有成本,這個算法是正確的嗎?",推推熊,目前遠東新大概跌到25.XX元~很想買....<BR><BR><BR>之前有有網友說他201...,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-04
40,1451842787160_F01,forum,mobile01,投資與理財(生活娛樂),2,2016-01-04 01:34:00,短期儲蓄的建議...,lance831005,因小弟有儲蓄的需求<BR>希望是短期(3~5年內)能夠存到約30萬<BR>今年的定存利率似乎...,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-04
41,1451862563673_F01,forum,mobile01,投資與理財(生活娛樂),4,2016-01-04 07:08:00,為何宇昌（中裕）換了CEO 股價就大漲呢,翠玉白菜888,宇昌生技 現改名為中裕新藥<BR>董事長由現在的總統後選人蔡英文 退出經營權 賣給尹衍樑<B...,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-04
42,1451871521490_F01,forum,mobile01,投資與理財(生活娛樂),56,2016-01-04 09:35:00,新年大爆跌......唉！,jacky8000,台股線型又要往下了，<BR>說好的總統行情呢......<BR>越接近大選越是跌,http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-04
43,1451877712033_F01,forum,mobile01,投資與理財(生活娛樂),21,2016-01-04 11:07:00,昨晚家人問我要不要多買點美金?,unl7911,"因為她們覺得匯率將來有可能來到 40:1, 大家覺得呢? Thanks.",http://www.mobile01.com/topicdetail.php?f=291&...,2016-01-04


In [28]:
for index, row in df_2327.iterrows():
    try:
        if df_2327.loc[index,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[index-1,'年月日'].date():                     
#            print(df_2327.loc[index-1,'年月日'].date())   
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            # 如果股票前一筆差1天  # 那前1天的文章標上當天的漲跌   
        
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-2) == df_2327.loc[index-1,'年月日'].date():
#            print(df_2327.loc[index-1,'年月日'].date())            
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            # 如果股票前一筆差2天 #那前2天的文章標上當天的漲跌
            
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-3) == df_2327.loc[index-1,'年月日'].date():
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-3,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-4) == df_2327.loc[index-1,'年月日'].date():
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-3,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-4,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-5) == df_2327.loc[index-1,'年月日'].date():
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-3,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-4,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-5,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-6) == df_2327.loc[index-1,'年月日'].date():
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-3,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-4,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-5,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-6,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-7) == df_2327.loc[index-1,'年月日'].date():
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-3,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-4,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-5,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-6,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-7,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-8) == df_2327.loc[index-1,'年月日'].date():
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-3,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-4,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-5,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-6,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-7,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-8,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-9) == df_2327.loc[index-1,'年月日'].date():
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-3,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-4,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-5,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-6,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-7,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-8,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-9,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
        elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-10) == df_2327.loc[index-1,'年月日'].date():
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-1,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-2,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-3,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-4,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-5,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-6,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-7,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-8,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-9,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
            df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index-10,'年月日'].date(), 'label'] = df_2327.loc[index,'tag']
    except:
        continue

In [29]:
len(df_forum[df_forum['label']=='down'])

2315

In [30]:
len(df_forum[df_forum['label']=='up'])

3988